<a href="https://colab.research.google.com/github/vibhu18116/IR-Project-Group23-PatentPal/blob/main/Summary/BigPatent_Baseline_Pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
data = pickle.load(open("/content/drive/MyDrive/bigPatentData/data_pik", 'rb'))

In [ ]:
import pandas as pd
df = pd.DataFrame(data[400:500])

In [ ]:
df

,publication_number,abstract,application_number,description
400,US-5091993-A,an insect proof garment for protection against...,US-62067090-A,referring now to the drawings and in particula...
401,US-2007294894-A1,the present invention discloses a one piece fo...,US-82331907-A,in the following detailed description of embod...
402,US-5388549-A,methods and apparatus enabling a diagnostic te...,US-10478993-A,the method and apparatus in accordance with th...
403,US-2013085319-A1,"a pump for inducing motion of a fluid , the pu...",US-201213590488-A,the present disclosure relates to novel and ad...
404,US-2009049993-A1,a cooking apparatus for batter based foods tha...,US-89249607-A,the preferred embodiments and best modes of th...
...,...,...,...,...
495,US-6434410-B1,an electrophysiological electrode includes mul...,US-10029098-A,"referring to fig1 - 3 , a preferred embodiment..."
496,US-6000158-A,a removable ironing board enclosed in a piece ...,US-13527698-A,"referring now the drawings , and in particular..."
497,US-2005131519-A1,a composite stent graft has a balloon expandab...,US-96276604-A,"now looking more closely at fig1 and 2 , it wi..."
498,US-2013227789-A1,"an apparatus and method for moving people , pe...",US-201213597990-A,"referring to fig1 , a rescue / recovery chair ..."


In [ ]:
!pip install torch==1.2.0 torchvision==0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu92/torch-1.2.0%2Bcu92-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.2.0+cu92 which is incompatible.
ERROR: sentence-transformers 1.1.0 has requirement torch>=1.6.0, but you'll have torch 1.2.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1


In [ ]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.1.0)
  Using cached https://files.pythonhosted.org/packages/56/74/6fc9dee50f7c93d6b7d9644554bdc9692f3023fa5d1de779666e6bf8ae76/torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchvision 0.4.0+cu92 has requirement torch==1.2.0, but you'll have torch 1.8.1 which is incompatible.
  Found existing installation: torch 1.2.0+cu92
    Uninstalling torch-1.2.0+cu92:
      Successfully uninstalled torch-1.2.0+cu92


In [ ]:
!pip install transformers

In [ ]:
!pip install rouge_score

In [ ]:
import random
from rouge_score import rouge_scorer

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True):
    #Calculate rouge scores
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    
    n = len(pred_summaries)
    
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
    
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

LSTM output as Pegasus Input

In [ ]:
with open('lstm-500-summary.pkl', 'rb') as f:
    df_lstm = pickle.load(f)

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import torch
# src_text = [
#     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
# ]

# src_text = df['description'][0]
model_name = 'google/pegasus-big_patent'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


In [ ]:
myfile = open('lstm-pegasus-opt.txt', 'w')

In [ ]:
for i in df_lstm['summaries_comp']:
    src_text = i
    batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest', return_tensors='pt').to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(tgt_text)
    myfile.write("%s\n" % tgt_text[0])
myfile.close()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['The garment is deliberately made loose fitting to enable the user to comfortably wear the garment over regular clothes, and to produce the blousing effect necessary for maximum insect protection properties.', 'An insect proof garment for protection against mosquitoes, black flies and other insects is disclosed. The garment is made entirely from a lightweight semi- rigid insect excluding mesh and is comprised of a one-piece fully encloseable head net attached to an upper body portion.']
['An aspect of the invention design has a curved triangular core extraction hook that enables the inside to be pulled out of the food object after the core flesh has been completely severed, core removal without ripping flesh.', 'The present invention discloses a one piece food corer with a hook design providing an efficient and elegant tool for coring food, hollowing out the flesh from a vegetable or fruit.']
['A method and apparatus for sampling milk from an animal being milked in which a pulsating v

In [ ]:
f = open("lstm-pegasus-opt.txt", "r")
pred_summaries = f.readlines()

In [ ]:
import numpy as np
scores_pegasus_lstm = calc_rouge_scores(pred_summaries, df['abstract'].tolist())

In [ ]:
scores_pegasus_lstm

{'rouge1': {'f1': 0.27108957999231276,
  'precision': 0.19935663483023824,
  'recall': 0.4914094516591353},
 'rouge2': {'f1': 0.08205525128368986,
  'precision': 0.05886296427393539,
  'recall': 0.15640260660832556},
 'rougeL': {'f1': 0.1852952000683844,
  'precision': 0.13508567989170275,
  'recall': 0.34295852086364387}}